In [5]:
using LinearAlgebra
using Printf
using Glob
using XLSX
using DataFrames
using Tables
using Dates

# Define una estructura para representar un vector 3D
struct Vector3D
    x::Float64
    y::Float64
    z::Float64
end

# Función para analizar una línea del archivo y extraer el vector
function parse_line(line::String)::Vector3D
    start_idx = findfirst('(', line) + 1
    end_idx = findfirst(')', line) - 1
    vector_str = line[start_idx:end_idx]
    tokens = split(vector_str, ',')

    x = parse(Float64, tokens[1])
    y = parse(Float64, tokens[2])
    z = parse(Float64, tokens[3])

    return Vector3D(x, y, z)
end

# Función para leer vectores desde un archivo
function read_vectors_from_file(filename::String)::Vector{Vector3D}
    vectors = Vector{Vector3D}()
    open(filename, "r") do file
        for line in eachline(file)
            try
                push!(vectors, parse_line(line))
            catch e
                println("Error parsing line: ", e)
            end
        end
    end
    return vectors
end

# Función para calcular la diferencia de fase entre dos vectores
function calculate_phase_difference(a::Vector3D, b::Vector3D)::Float64
    dot_product = a.x * b.x + a.y * b.y + a.z * b.z
    magnitude_a = norm([a.x, a.y, a.z])
    magnitude_b = norm([b.x, b.y, b.z])
    cos_theta = dot_product / (magnitude_a * magnitude_b)

    cos_theta = clamp(cos_theta, -1.0, 1.0)

    return acos(cos_theta)
end

# Modelo de Kuramoto y cálculo de energía
function kuramoto_model(vectors::Vector{Vector3D}, K::Float64)::Float64
    N = length(vectors)
    total_energy = 0.0

    Threads.@threads for i in 1:N
        for j in 1:N
            if i != j
                phase_diff = calculate_phase_difference(vectors[i], vectors[j])
                total_energy += K * sin(phase_diff)
            end
        end
    end

    return total_energy
end

# Función para guardar los resultados en un archivo Excel
function save_results_to_excel(results::DataFrame, output_filename::String)
    XLSX.openxlsx(output_filename, mode="w") do workbook
        sheet = XLSX.addsheet!(workbook, "Resultados")
        XLSX.writetable!(sheet, Tables.columntable(results), anchor_cell=XLSX.CellRef("A1"))
    end
end

# Función principal para procesar los archivos según la estructura definida
function process_data(root_folder::String)
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]
    results = DataFrame(date = String[], shift = String[], lab = String[], color = String[], total_energy = Float64[])  # Variable para almacenar resultados

    for lab in labs
        for shift in shifts
            # Buscar el directorio del laboratorio en la carpeta raíz
            lab_dirs = glob("*$lab*", root_folder)
            if isempty(lab_dirs)
                println("No se encontró el directorio para $lab en $root_folder")
                continue
            end
            
            lab_dir = lab_dirs[1]
            println("Directorio encontrado para $lab: $lab_dir")
            
            date_str = match(r"\d{2}\w{3}\d{2}", lab_dir)
            if date_str === nothing
                println("No se pudo extraer la fecha del nombre del directorio $lab_dir")
                continue
            end
            date_str = date_str.match
            
            println("Fecha extraída: $date_str, Turno: $shift")

            # Definir las rutas de las subcarpetas
            baseline_dir = joinpath(lab_dir, "$date_str.$shift - $lab", "Data Analysis", "Processing Data", "Curl_Baseline")
            exp_dir = joinpath(lab_dir, "$date_str.$shift - $lab", "Data Analysis", "Processing Data", "Curl_Experimental_Color")
            
            println("Ruta de Curl_Baseline: $baseline_dir")
            println("Ruta de Curl_Experimental_Color: $exp_dir")

            if !(isdir(baseline_dir) && isdir(exp_dir))
                println("No se encontraron las subcarpetas necesarias para $lab en $shift")
                continue
            end

            baseline_files = glob("*mednegra.txt", baseline_dir) |> filter(x -> !occursin("00", x))
            experimental_files = glob("*.txt", exp_dir) |> filter(x -> occursin(r"med(roja|morada|verde|azul).txt", x) && !occursin("00", x))

            println("Archivos en $baseline_dir:")
            println(baseline_files)
            println("Archivos en $exp_dir:")
            println(experimental_files)
            
            # Procesamiento de los archivos encontrados y almacenamiento de resultados
            for file in baseline_files
                vectors = read_vectors_from_file(file)
                total_energy = kuramoto_model(vectors, 1.0)
                color = "negra"
                push!(results, (date_str, shift, lab, color, total_energy))
            end

            for file in experimental_files
                vectors = read_vectors_from_file(file)
                total_energy = kuramoto_model(vectors, 1.0)
                color_match = match(r"med(\w+).txt", file)
                color = color_match !== nothing ? color_match.captures[1] : "unknown"
                push!(results, (date_str, shift, lab, color, total_energy))
            end
        end
    end
    
    # Extraer la fecha del nombre de la carpeta root_folder, asumiendo un formato similar a "29Ago24"
    # Esto depende del formato exacto de tu carpeta; ajusta la expresión regular si es necesario
    folder_date_match = match(r"\d{2}\w{3}\d{2}", basename(root_folder))

    # Verificar si se encontró una fecha en el nombre de la carpeta
    if folder_date_match !== nothing
    folder_date = folder_date_match.match
    else
    println("No se pudo extraer la fecha del nombre del directorio root_folder. Se usará 'SinFecha'.")
    folder_date = "SinFecha"
    end

    # Guardar los resultados en un archivo Excel en el root_folder con la fecha en el nombre del archivo
    output_filename = joinpath(root_folder, "Total_Energy_$folder_date.xlsx")
    save_results_to_excel(results, output_filename)
    println("Resultados guardados en: $output_filename")
end

# Llamada a la función principal con la ruta del directorio raíz
root_folder = raw"E:\Septiembre\07Sep24"
process_data(root_folder)

Directorio encontrado para Lab_Betta: E:\Septiembre\07Sep24\07Sep24 - Lab_Betta
Fecha extraída: 07Sep24, Turno: V
Ruta de Curl_Baseline: E:\Septiembre\07Sep24\07Sep24 - Lab_Betta\07Sep24.V - Lab_Betta\Data Analysis\Processing Data\Curl_Baseline
Ruta de Curl_Experimental_Color: E:\Septiembre\07Sep24\07Sep24 - Lab_Betta\07Sep24.V - Lab_Betta\Data Analysis\Processing Data\Curl_Experimental_Color
No se encontraron las subcarpetas necesarias para Lab_Betta en V
Directorio encontrado para Lab_Betta: E:\Septiembre\07Sep24\07Sep24 - Lab_Betta
Fecha extraída: 07Sep24, Turno: M
Ruta de Curl_Baseline: E:\Septiembre\07Sep24\07Sep24 - Lab_Betta\07Sep24.M - Lab_Betta\Data Analysis\Processing Data\Curl_Baseline
Ruta de Curl_Experimental_Color: E:\Septiembre\07Sep24\07Sep24 - Lab_Betta\07Sep24.M - Lab_Betta\Data Analysis\Processing Data\Curl_Experimental_Color
Archivos en E:\Septiembre\07Sep24\07Sep24 - Lab_Betta\07Sep24.M - Lab_Betta\Data Analysis\Processing Data\Curl_Baseline:
["E:\\Septiembre\\07Se

GENERACIÓN DE ESTADISTICAS: 

